In [ ]:
import re
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from old_functions import *

Players Career Data

In [46]:
soup = get_soup('https://www.basketball-reference.com/players/a/antetgi01.html')

In [93]:

### Individual Per Game data: Regular Season and Playoff

def player_avg_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    per_game_data = soup.select('table[data-soc-sum-table-type="PlayerPerGame"]')

    if not per_game_data:
        print('No Data Found')
        return

    ### Regular Season Table

    per_game_reg_season_columns = per_game_data[0].select('th[scope="col"]')
    per_game_reg_season_columns = [
        col.get_text(strip=True) for col in per_game_reg_season_columns
    ]

    per_game_reg_season_table_body = per_game_data[0].select('tbody')[0].select('th,td')
    table_data = []
    for i in per_game_reg_season_table_body:
        table_data.append(i.get_text(strip=True))

    reg_season_data = []
    BATCH_SIZE = 31
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        reg_season_data.append(curr)

    reg_season = pd.DataFrame(reg_season_data)
    reg_season.columns = per_game_reg_season_columns

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoff Table

    try:
        per_game_playoff_columns = per_game_data[1].select('th[scope="col"]')
    except IndexError as e:
        print('No Data Found')

    per_game_playoff_columns = [
        col.get_text(strip=True) for col in per_game_playoff_columns
    ]

    per_game_playoff_table_body = per_game_data[1].select('tbody')[0].select('th,td')
    table_data = []
    for i in per_game_playoff_table_body:
        table_data.append(i.get_text(strip=True))

    playoff_data = []
    BATCH_SIZE = 31
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        playoff_data.append(curr)

    playoff_season = pd.DataFrame(playoff_data)
    playoff_season.columns = per_game_playoff_columns

    playoff_season = playoff_season[
        playoff_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    playoff_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, playoff_season], axis=0)

    return data


In [88]:
def player_sum_data(player_username):
    
    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    total_data = soup.select('table[data-soc-sum-table-type="PlayerTotals"]')

    if not total_data:
        print('No Data Found')
        return

    ### Regular Season

    reg_data = total_data[0]
    reg_cols = [col.get_text() for col in reg_data.select('th[scope="col"]')]

    total_reg_season_table_body = reg_data.select('tbody')[0].select('th,td')
    table_data = []
    for i in total_reg_season_table_body:
        table_data.append(i.get_text(strip=True))

    reg_season_data = []
    BATCH_SIZE = len(reg_cols)
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        reg_season_data.append(curr)

    reg_season = pd.DataFrame(reg_season_data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = total_data[1]
    except IndexError as e:
        print('No Data Found')
        return reg_season
        
    post_cols = [col.get_text() for col in post_data.select('th[scope="col"]')]

    total_post_season_table_body = post_data.select('tbody')[0].select('th,td')
    table_data = []
    for i in total_post_season_table_body:
        table_data.append(i.get_text(strip=True))

    post_season_data = []
    BATCH_SIZE = len(post_cols)
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        post_season_data.append(curr)

    post_season = pd.DataFrame(post_season_data)
    post_season.columns = post_cols

    post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'
    
    data = pd.concat([reg_season, post_season], axis=0)

    return data

In [87]:
def player_per36_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    table_data = soup.select('table[data-soc-sum-table-type="PlayerPerMinute"]')

    if not table_data:
        print('No Data Found')
        return

    ### Regular season

    reg_data = table_data[0]
    reg_cols = [
        col.get_text(strip=True) for col in reg_data.select('th[scope="col"]')
    ]

    reg_data = [
        row
        .get_text(strip=True) for row in reg_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(reg_cols)
    data = []
    for i in range(0, len(reg_data), BATCH_SIZE):
        curr_data = reg_data[i: i+BATCH_SIZE]
        data.append(curr_data)

    reg_season = pd.DataFrame(data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = table_data[1]
    except IndexError as e:
        print("No Playoff Data")
        return reg_season

    post_cols = [
        col.get_text(strip=True) for col in post_data.select('th[scope="col"]')
    ]

    post_data = [
        row
        .get_text(strip=True) for row in post_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(post_cols)
    data = []
    for i in range(0, len(post_data), BATCH_SIZE):
        curr = post_data[i: i+BATCH_SIZE]
        data.append(curr)

    post_season = pd.DataFrame(data)
    post_season.columns = post_cols

    post_season = post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, post_season], axis=0)

    return data

In [86]:
def player_advanced_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    table_data = soup.select('table[data-soc-sum-table-type="Advanced"]')

    if not table_data:
        print('No Data Found')
        return

    ### Regular season

    reg_data = table_data[0]
    reg_cols = [
        col.get_text(strip=True) for col in reg_data.select('th[scope="col"]')
    ]

    reg_data = [
        row
        .get_text(strip=True) for row in reg_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(reg_cols)
    data = []
    for i in range(0, len(reg_data), BATCH_SIZE):
        curr_data = reg_data[i: i+BATCH_SIZE]
        data.append(curr_data)

    reg_season = pd.DataFrame(data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = table_data[1]
    except IndexError as e:
        print("No Playoff Data")
        return reg_season

    post_cols = [
        col.get_text(strip=True) for col in post_data.select('th[scope="col"]')
    ]

    post_data = [
        row
        .get_text(strip=True) for row in post_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(post_cols)
    data = []
    for i in range(0, len(post_data), BATCH_SIZE):
        curr = post_data[i: i+BATCH_SIZE]
        data.append(curr)

    post_season = pd.DataFrame(data)
    post_season.columns = post_cols

    post_season = post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, post_season], axis=0)

    return data

In [103]:
def player_shooting_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    table_data = soup.select('table[data-soc-sum-table-type="Shooting"]')

    if not table_data:
        print('No Data Found')
        return

    ### Regular season

    reg_data = table_data[0]
    reg_cols = [
        col.get_text(strip=True) for col in reg_data.select('th[scope="col"]')
    ]

    reg_data = [
        row
        .get_text(strip=True) for row in reg_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(reg_cols)
    data = []
    for i in range(0, len(reg_data), BATCH_SIZE):
        curr_data = reg_data[i: i+BATCH_SIZE]
        data.append(curr_data)

    reg_season = pd.DataFrame(data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = table_data[1]
    except IndexError as e:
        print("No Playoff Data")
        return reg_season

    post_cols = [
        col.get_text(strip=True) for col in post_data.select('th[scope="col"]')
    ]

    post_data = [
        row
        .get_text(strip=True) for row in post_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(post_cols)
    data = []
    for i in range(0, len(post_data), BATCH_SIZE):
        curr = post_data[i: i+BATCH_SIZE]
        data.append(curr)

    post_season = pd.DataFrame(data)
    post_season.columns = post_cols

    post_season = post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'

    shooting_data = pd.concat([reg_season, post_season], axis=0)

    return shooting_data

Player Season Data

In [104]:
soup = get_soup('https://www.basketball-reference.com/players/a/antetgi01/gamelog/2023/')

In [208]:
def player_season_data(player_username, season):    
    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}/gamelog/{season}/')
    
    table = soup.select_one('table#player_game_log_reg')

    if not table:
        print("No Data Found")
        return

    header_cells = table.select('thead tr')[-1].select('th[scope="col"]')
    cols = [c.get_text(strip=True) for c in header_cells]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    reg_season = pd.DataFrame(rows, columns=cols)
    reg_season['Season Type'] = 'Regular'

    ### Playoffs

    table = soup.select_one('table#player_game_log_post')

    if not table:
        print('No Playoff Data Found')

        reg_season = reg_season.drop(columns='')

        reg_season['TS'] = reg_season['Result'].apply(lambda x: re.findall(' [0-9]*', x)[0]).str.replace(' ', '')
        reg_season['OS'] = reg_season['Result'].apply(lambda x: re.findall('-[0-9]*', x)[0]).str.replace('-', '')
        reg_season['Result'] = reg_season['Result'].apply(lambda x: x[0])

        return reg_season
    
    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    post_season = pd.DataFrame(rows, columns=cols)
    post_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, post_season], axis=0)
    data = data.drop(columns='')

    data['TS'] = data['Result'].apply(lambda x: re.findall(' [0-9]*', x)[0]).str.replace(' ', '')
    data['OS'] = data['Result'].apply(lambda x: re.findall('-[0-9]*', x)[0]).str.replace('-', '')
    data['Result'] = data['Result'].apply(lambda x: x[0])

    return data

In [211]:
player_data = player_season_data('murrake02', '2025')

No Playoff Data Found


Team Data

In [2]:
soup = get_soup('https://www.basketball-reference.com/teams/')

In [43]:
teams = soup.select_one('table#teams_active')

In [ ]:
def all_team_historical_data():
    soup = get_soup('https://www.basketball-reference.com/teams/')

    teams = soup.select_one('table#teams_active')

    cols = [
        col.get_text(strip=True) for col in teams.select('th[scope="col"]')
    ]

    NUM_COLS = len(cols)
    rows = []
    for tr in teams.select('tbody tr'):
        if 'left' in tr.select_one('th')['class']:
            continue
        
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)
        
    teams = pd.DataFrame(rows, columns=cols)

    return teams

In [74]:
soup = get_soup('https://www.basketball-reference.com/teams/ATL/')

if not soup.select_one('table#SAC'):
    print("hello")

hello


In [ ]:
def team_historyical_data(team):
    soup = get_soup(f'https://www.basketball-reference.com/teams/{team}/')

    table = soup.select_one(f'table#{team}')

    if not table:
        print('No Data Found')
        return

    cols = [col.get_text(strip=True) for col in table.select('th[scope="col"]')]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue

        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)
        
    return pd.DataFrame(rows, columns=cols)

In [76]:
soup = get_soup('https://www.basketball-reference.com/teams/ATL/2025.html')

In [77]:
def team_season_data(team, year):
    soup = get_soup(
        f'https://www.basketball-reference.com/teams/{team}/{year}.html'
    )

    table = soup.select_one('table#roster')

    if not table:
        print('No Data Found')
        return

    cols = [
        col.get_text(strip=True) for col in table.select('th[scope="col"]')
    ]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    return pd.DataFrame(rows, columns=cols)


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College
0,0,Dominick Barlow,SF,6-9,221,"May 26, 2003",usUS,2,
1,13,Bogdan Bogdanović,SG,6-5,220,"August 18, 1992",rsRS,7,
2,4,Kobe Bufkin,SG,6-4,195,"September 21, 2003",usUS,1,Michigan
3,15,Clint Capela,C,6-10,240,"May 18, 1994",chCH,10,
4,5,Dyson Daniels,SG,6-8,199,"March 17, 2003",auAU,2,
5,18,Mouhamed Gueye,PF,6-11,210,"November 9, 2002",snSN,1,Washington State
6,12,De'Andre Hunter,SF,6-8,225,"December 2, 1997",usUS,5,Virginia
7,1,Jalen Johnson,SF,6-9,220,"December 18, 2001",usUS,3,Duke
8,27,Vit Krejci,PG,6-8,195,"June 19, 2000",czCZ,3,
9,3,Caris LeVert,SG,6-6,205,"August 25, 1994",usUS,8,Michigan


In [ ]:
def team_avg_data(team, year):
    soup = get_soup(
        f'https://www.basketball-reference.com/teams/{team}/{year}.html'
    )
    
    table = soup.select_one('table#per_game_stats')

    if not table:
        print('No Data Found')
        return 

    cols = [col.get_text(strip=True) for col in table.select('th[scope="col"]')]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    return pd.DataFrame(rows, columns=cols)

In [91]:
def team_per36_data(team, year):
    soup = get_soup(
        f'https://www.basketball-reference.com/teams/{team}/{year}.html'
    )
    
    table = soup.select_one('table#per_minute_stats')

    if not table:
        print('No Data Found')
        return 

    cols = [col.get_text(strip=True) for col in table.select('th[scope="col"]')]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    return pd.DataFrame(rows, columns=cols)


team_per36_data('SAC', '2025')

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,1,DeMar DeRozan,35,SF,77,77,2768,8.1,17.0,.477,...,0.7,3.2,3.9,4.4,0.8,0.4,1.4,2.0,22.2,
1,2,Keegan Murray,24,PF,76,76,2610,5.0,11.3,.444,...,2.0,5.0,7.0,1.5,0.8,1.0,0.8,2.6,13.0,
2,3,Domantas Sabonis,28,C,70,70,2429,7.8,13.2,.590,...,4.0,10.4,14.4,6.3,0.7,0.4,3.0,3.4,19.8,
3,4,Malik Monk,26,SG,65,45,2054,7.2,16.5,.439,...,0.5,3.8,4.3,6.3,1.1,0.7,2.8,2.8,19.6,
4,5,Keon Ellis,25,SG,80,28,1948,4.2,8.6,.489,...,1.1,2.8,3.9,2.2,2.2,1.2,1.3,3.3,12.3,
5,6,De'Aaron Fox,27,PG,45,45,1663,9.0,19.1,.469,...,1.0,3.9,4.9,6.0,1.4,0.4,2.9,2.6,24.3,
6,7,Trey Lyles,29,PF,69,5,1349,4.2,10.0,.420,...,2.1,6.3,8.4,2.2,1.1,0.6,1.0,2.3,12.0,
7,8,Zach LaVine,29,SG,32,32,1170,8.1,15.8,.511,...,0.3,3.2,3.5,3.7,0.6,0.1,2.7,1.8,22.1,
8,9,Kevin Huerter,26,SG,43,15,899,5.1,12.4,.413,...,0.8,4.0,4.8,3.0,1.4,0.6,1.5,2.8,13.5,
9,10,Jonas Valančiūnas,32,C,32,9,541,7.6,13.6,.556,...,4.7,10.2,14.8,3.7,1.0,1.2,2.1,5.6,18.6,


In [101]:
soup = get_soup(
    f'http://basketball-reference.com/leagues/'
)

In [100]:
soup.select_one('table#per_minute_stats')

<table class="stats_table sortable soc" data-cols-to-freeze=",2" data-soc-sum-params="null" data-soc-sum-phase-type="reg" data-soc-sum-scope-type="player_season" data-soc-sum-year="2025" id="per_minute_stats"> <caption>Per 36 Minutes Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rk" class="ranker poptip center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th> <th aria-label="Player" class="poptip sort_default_asc center" data-stat="name_display" scope="col">Player</th> <th aria-label="Age" class="poptip center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th> <th aria-label="Pos" class="poptip center" data-stat="pos" data-tip="Position" scope="col">Pos</th> <th aria-label="G" class="poptip center" data-stat="games" data-tip="Games" scope="col"

In [120]:
def season_data():
    soup = get_soup(
        f'http://basketball-reference.com/leagues/'
    )
    
    table = soup.select_one('table#stats')

    if not table:
        print('No Data Found')
        return

    cols = [col.get_text(strip=True) for col in table.select('th[class="sort_default_asc center"]')]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue

        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    return pd.DataFrame(rows, columns=cols)



In [ ]:
sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import Scrape_Functions

In [175]:
soup = get_soup('https://www.basketball-reference.com/players/m/murrake02.html')

In [176]:
table = soup.select_one('table[data-soc-sum-table-type="PlayerPerGame"][data-soc-sum-phase-type="reg"]')

cols = [col.get_text(strip=True) for col in table.select_one('tr').select('th')]
NUM_COLS = len(cols)

rows = []
for tr in table.select('tbody tr'):
    if 'class' in tr.attrs and 'thead' in tr['class']:
        continue

    cells = tr.find_all(['th', 'td'])
    row = [c.get_text(strip=True) for c in cells]
    if len(row) < NUM_COLS:
        row += [''] * (NUM_COLS - len(row))
    elif len(row) > NUM_COLS:
        row = row[:NUM_COLS]
    rows.append(row)

reg_season = pd.DataFrame(rows, columns=cols)

reg_season = reg_season[
    reg_season.apply(lambda x: x != '')
].dropna(how='all', axis=0).fillna('')

table = soup.select_one('table[data-soc-sum-table-type="PlayerPerGame"][data-soc-sum-phase-type="post"]')

cols = [col.get_text(strip=True) for col in table.select_one('tr').select('th')]
NUM_COLS = len(cols)

rows = []
for tr in table.select('tbody tr'):
    if 'class' in tr.attrs and 'thead' in tr['class']:
        continue

    cells = tr.find_all(['th', 'td'])
    row = [c.get_text(strip=True) for c in cells]
    if len(row) < NUM_COLS:
        row += [''] * (NUM_COLS - len(row))
    elif len(row) > NUM_COLS:
        row = row[:NUM_COLS]
    rows.append(row)

playoff_season = pd.DataFrame(rows, columns=cols)

playoff_season = playoff_season[
    playoff_season.apply(lambda x: x != '')
].dropna(how='all', axis=0).fillna('')
    

reg_season['Season Type'] = 'Regular'
playoff_season['Season Type'] = 'Playoff'

data = pd.concat([reg_season, playoff_season], axis=0)
data = data.reset_index(drop=True)

data

,Season,Age,Team,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Season Type
0,2022-23,22,SAC,NBA,SF,80,78,29.8,4.4,9.8,...,3.5,4.6,1.2,0.8,0.5,0.8,2.0,12.2,ROY-5,Regular
1,2023-24,23,SAC,NBA,SF,77,77,33.6,5.8,12.7,...,4.1,5.5,1.7,1.0,0.8,0.8,2.7,15.2,,Regular
2,2024-25,24,SAC,NBA,PF,76,76,34.3,4.8,10.8,...,4.8,6.7,1.4,0.8,0.9,0.8,2.4,12.4,,Regular
3,2022-23,22,SAC,NBA,SF,7,7,27.7,3.7,8.3,...,4.1,6.3,0.7,0.6,0.3,0.6,2.6,9.7,,Playoff
